In [40]:
from turtle import back
from skimage.io import imread, imsave
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier

parent_path =r'D:\images\tnia-python-images\\imagesc\\2024_12_19_sem_grain_size_revisit'
training_data_path = os.path.join(parent_path,r'training_data')

im_name = '211122_AM_Al2O3_SE_021.ome.tiff'
validation_name = r'211122_AM_Al2O3_SE_027_sp.tif'

labels_name = 'brian_labels.tiff'

In [41]:
im = imread(os.path.join(training_data_path,im_name))
labels = imread(os.path.join(training_data_path,labels_name))
predictions = np.zeros_like(labels)

validation_im = imread(os.path.join(training_data_path,validation_name))

In [43]:
import napari 
viewer = napari.Viewer()
viewer.add_image(im)
viewer.add_labels(labels, name='labels')
viewer.add_labels(predictions, name='predictions')

<Labels layer 'predictions' at 0x20d303af0a0>

In [44]:
from tnia.machinelearning.random_forest_helper import extract_features


In [45]:
features = extract_features(im)
print(features.shape)

(1536, 2048, 12)


In [46]:
label_vector = labels[labels>0]
features_vector = features[labels>0,:]
print(label_vector.shape, features_vector.shape)

clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10, max_samples=0.05)
clf.fit(features_vector, label_vector-1)


(7270,) (7270, 12)


RandomForestClassifier(max_depth=10, max_samples=0.05, n_estimators=50,
                       n_jobs=-1)

In [47]:
import numpy as np
from skimage import future


predictions = future.predict_segmenter(features.reshape(-1, features.shape[-1]), clf).reshape(features.shape[:-1]) + 1
predictions = np.squeeze(predictions).astype(np.uint32)
print(predictions.min(), predictions.max())
viewer.layers['predictions'].data = predictions

1 2


In [48]:
predictions.min(), predictions.max(), predictions.dtype

(1, 2, dtype('uint32'))

In [49]:
from skimage.measure import label

instances = label(predictions==2)

viewer.add_labels(instances, name='labeled grains')


<Labels layer 'labeled grains' at 0x20d0a9e37c0>

In [50]:
validation_features = extract_features(validation_im)
validation_predictions = future.predict_segmenter(validation_features.reshape(-1, validation_features.shape[-1]), clf).reshape(validation_features.shape[:-1]) + 1

validation_labels = label(validation_predictions==2)
viewer.add_labels(validation_predictions, name='validation predictions')
viewer.add_labels(validation_labels, name='validation labeled grains')


<Labels layer 'validation labeled grains' at 0x20d157914e0>